In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datasets import load_dataset
from collections import Counter
from conlleval import evaluate

**TransformerBlock**

In [2]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

**Embedding Layer**

In [3]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings

**NER Model**

In [4]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        super(NERModel, self).__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout1 = layers.Dropout(0.1)
        self.ff = layers.Dense(ff_dim, activation="relu")
        self.dropout2 = layers.Dropout(0.1)
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x

In [5]:
conll_data = load_dataset("conll2003")

Reusing dataset conll2003 (C:\Users\dongw\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee)
100%|██████████| 3/3 [00:00<00:00, 111.06it/s]


In [6]:
def export_to_file(export_file_path, data):
    with open(export_file_path, "w") as f:
        for record in data:
            ner_tags = record["ner_tags"]
            # for i in range(len(ner_tags)):
            #     if ner_tags[i] in {8, 9}:
            #         ner_tags[i] = 1
            tokens = record["tokens"]
            if len(tokens) > 0:
                f.write(
                    str(len(tokens))
                    + "\t"
                    + "\t".join(tokens)
                    + "\t"
                    + "\t".join(map(str, ner_tags))
                    + "\n"
                )


if not os.path.isdir('data'):
    os.mkdir("data")
export_to_file("./data/conll_train.txt", conll_data["train"])
export_to_file("./data/conll_val.txt", conll_data["validation"])

In [7]:
def make_tag_lookup_table():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    res_dict = dict(zip(range(0, len(all_labels) + 1), all_labels))
    return res_dict


mapping = make_tag_lookup_table()
print(mapping)

{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}


In [8]:
all_tokens = sum(conll_data["train"]["tokens"], [])
all_tokens_array = np.array(list(map(str.lower, all_tokens)))

counter = Counter(all_tokens_array)
print(len(counter))

num_tags = len(mapping)
vocab_size = 20000

vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]

# Convert tokens to token IDs
lookup_layer = keras.layers.StringLookup(
    vocabulary=vocabulary
)

21009


In [9]:
train_data = tf.data.TextLineDataset("./data/conll_train.txt")
val_data = tf.data.TextLineDataset("./data/conll_val.txt")

In [10]:
def map_record_to_training_data(record):
    record = tf.strings.split(record, sep="\t")
    length = tf.strings.to_number(record[0], out_type=tf.int32)
    tokens = record[1 : length + 1]
    tags = record[length + 1 :]
    tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags += 1
    return tokens, tags


def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)


# We use `padded_batch` here because each record in the dataset has a
# different length.
batch_size = 32
train_dataset = (
    train_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)
val_dataset = (
    val_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)

ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)

In [11]:
class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)


loss = CustomNonPaddingTokenLoss()

In [12]:
ner_model.compile(optimizer="adam", loss=loss)
ner_model.fit(train_dataset, epochs=10)

Epoch 1/10


c:\Users\dongw\miniconda3\envs\ml\lib\site-packages\keras\backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


439/439 [==============================] - 10s 16ms/step - loss: 0.5962
Epoch 2/10
439/439 [==============================] - 7s 15ms/step - loss: 0.2484
Epoch 3/10
439/439 [==============================] - 7s 15ms/step - loss: 0.1496
Epoch 4/10
439/439 [==============================] - 6s 14ms/step - loss: 0.1148
Epoch 5/10
439/439 [==============================] - 6s 14ms/step - loss: 0.0923
Epoch 6/10
439/439 [==============================] - 6s 14ms/step - loss: 0.0761
Epoch 7/10
439/439 [==============================] - 7s 16ms/step - loss: 0.0649
Epoch 8/10
439/439 [==============================] - 7s 15ms/step - loss: 0.0541
Epoch 9/10
439/439 [==============================] - 7s 15ms/step - loss: 0.0477
Epoch 10/10
439/439 [==============================] - 8s 17ms/step - loss: 0.0429


In [13]:
def calculate_metrics(dataset):
    all_true_tag_ids, all_predicted_tag_ids = [], []

    for x, y in dataset:
        output = ner_model.predict(x)
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        true_tag_ids = np.reshape(y, [-1])

        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]

    evaluate(real_tags, predicted_tags)


calculate_metrics(val_dataset)

processed 51362 tokens with 5942 phrases; found: 5748 phrases; correct: 4024.
accuracy:  65.49%; (non-O)
accuracy:  93.40%; precision:  70.01%; recall:  67.72%; FB1:  68.85
              LOC: precision:  81.53%; recall:  82.20%; FB1:  81.87  1852
             MISC: precision:  71.74%; recall:  64.97%; FB1:  68.18  835
              ORG: precision:  58.25%; recall:  63.98%; FB1:  60.98  1473
              PER: precision:  66.56%; recall:  57.38%; FB1:  61.63  1588


In [14]:
def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)

tf.Tensor([[  988 10950   204   628     6  3938   215  5773]], shape=(1, 8), dtype=int64)
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O']


In [15]:
ner_model.save('ner.model')

INFO:tensorflow:Assets written to: ner.model\assets


INFO:tensorflow:Assets written to: ner.model\assets


In [16]:
def convert_iob_to_entities(tokens, iobs):
    length = len(tokens)
    entities = []
    entity_types = []
    entity_type = None
    entity = []
    entity_dict = {'LOC': 'address', 'ORG': 'organization', 'PER': 'person'}
    for i in range(length):
        temp = iobs[i].split('-')
        if len(temp) == 1:
            continue
        symbol = temp[0]
        e_type = temp[1]
        if e_type in entity_dict:
            entity.append(tokens[i])
            entity_type = entity_dict[e_type]
            if i == length-1 or iobs[i+1] == 'O' or iobs[i+1][0] == 'B':
                entities.append(' '.join(entity))
                entity_types.append(entity_type)
                entity = []
                entity_type = None
    return entities, entity_types


def get_entities(sent, model):
    input = tokenize_and_convert_to_ids(sent)
    input = tf.reshape(input, shape=[1, -1])
    output = model.predict(input)
    prediction = np.argmax(output, axis=-1)[0]
    prediction = [mapping[i] for i in prediction]
    entities, entity_types = convert_iob_to_entities(sent.split(), prediction)
    return entities, entity_types

(['eu'], ['organization'])

**Test sample**

In [ ]:
sample_input = "eu rejects german call to boycott british lamb."
get_entities(sample_input, ner_model)

**Recognize entities in database**

In [22]:
import sqlite3
from database import DATABASE_NAME
con = sqlite3.connect(DATABASE_NAME)
cur = con.cursor()
cur.execute('''
SELECT rowid, sent from sentence;
''')
rows = cur.fetchall()
for id, sent in rows:
    records = []
    entities, entity_types = get_entities(sent, ner_model)
    for entity, entity_type in zip(entities, entity_types):
        records.append((entity, entity_type, id))
    cur.executemany('''
    INSERT INTO ner (entity, entity_type, sentence_id)
    VALUES(?, ?, ?);''', records)
con.commit()
con.close()